In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

num_classes=131
img_rows,img_cols=100,100
batch_size=16

train_data_dir="D:/fruits-360/Training/"
validation_data_dir="D:/fruits-360/validation/"

train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=30,
                                 width_shift_range=0.3,
                                 horizontal_flip=True,
                                 fill_mode='nearest')

validation_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(img_rows, img_cols),
                                                 batch_size=batch_size,
                                                 class_mode="categorical",
                                                 shuffle=True)

validation_generator=validation_datagen.flow_from_directory(validation_data_dir,
                                                           target_size=(img_rows, img_cols),
                                                           batch_size=batch_size,
                                                           class_mode="categorical",
                                                           shuffle=False)

Using TensorFlow backend.


Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [2]:
#lets define our model
model=Sequential()

model.add(Conv2D(32,(3,3),padding='same',
                input_shape=(img_rows,img_cols,3)))
model.add(Activation("relu"))
model.add(Conv2D(32,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation("softmax"))

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 98, 98, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 49, 32)        0         
______________________________________

In [3]:
#below can be done but gives error, maybe version issue can be fixed later
#from keras.utils.vis_utils import plot_model
#plot_model(model,show_shapes=True,show_layer_names=True)

In [4]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

checkpoint=ModelCheckpoint(r"D:/backup for python files/fruit_image_classifier_with_all_callbacks.h5",
                          monitor="val_loss",
                          mode="min",
                          save_best_only=True,
                          verbose=1)

earlystop=EarlyStopping(monitor="val_loss",
                       min_delta=0,
                       patience=3,
                       verbose=1,
                       restore_best_weights=True)

reduce_lr=ReduceLROnPlateau(monitor="val_loss",
                           factor=0.2,
                           patience=3,
                           min_delta=0.0001)

callbacks=[checkpoint,earlystop,reduce_lr]


model.compile(loss="categorical_crossentropy",
             optimizer=Adam(learning_rate=0.01),
              metrics=['accuracy'])


nb_train_samples=692
nb_validation_samples=688
epochs=3


history=model.fit_generator(train_generator,
                           steps_per_epoch=nb_train_samples // batch_size,
                           epochs=epochs,
                           callbacks=callbacks,
                           validation_data=validation_generator,
                           validation_steps=nb_validation_samples // batch_size)


Epoch 1/3
43/43 [==============================] - 714s 17s/step - loss: 7.7198 - accuracy: 0.0102 - val_loss: 5.1768 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 5.17676, saving model to D:/backup for python files/fruit_image_classifier_with_all_callbacks.h5
Epoch 2/3
43/43 [==============================] - 634s 15s/step - loss: 4.8800 - accuracy: 0.0087 - val_loss: 4.8151 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss improved from 5.17676 to 4.81506, saving model to D:/backup for python files/fruit_image_classifier_with_all_callbacks.h5
Epoch 3/3
43/43 [==============================] - 640s 15s/step - loss: 4.8719 - accuracy: 0.0131 - val_loss: 4.6649 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss improved from 4.81506 to 4.66491, saving model to D:/backup for python files/fruit_image_classifier_with_all_callbacks.h5


In [12]:
!pip install scikit-learn

In [ ]:
##DISPLAYING OUR CONFUSION MATRIX


import matplotlib.pyplot as plt

from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
from keras.models import load_model

img_row,img_height,img_depth=32,32,3
model=load_model("D:/backup for python files/fruit_image_classifier_with_all_callbacks.h5")

class_labels=validation_generator.class_indices
class_labels={v:k for k, v in class_labels.items() }
classes=list(class_labels.values())

nb_train_samples=692 
nb_validation_samples=688 

#confusion matrix and classification report
Y_pred=model.predict_generator(validation_generator,nb_validation_samples //batch_size)
y_pred=np.argmax(Y_pred,axis=1)

target_names=list(class_labels.values())

plt.figure(figsize=(20,20))
cnf_matrix=confusion_matrix(validation_generator.classes, y_pred)

plt.imshow(cnf_matrix,interpolation='nearest')
plt.colorbar()
tick_marks=np.arange(len(classes))
_=plt.xticks(tick_marks,classes, rotation=90)
_=plt.yticks(tick_names,classes)

In [14]:
from keras.preprocessing import image
import numpy as np
import os
import cv2
from os import listdir
from os.path import isfile, join
import re

def draw_test(name,pred,im,true_label):
    BLACK=[0,0,0]
    expanded_image=cv2.copyMakeBorder(im,160,0,0,500,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, "predicted-"+pred, (20,60),cv2.FONT_HERSHEY_SIMPLES,1,(0,0,255),2)
    cv2.putText(expanded_image,"true-"+true_label,(20,120),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow(name,expanded_image)
    
    

def getRandomImage(pathj, img_width, img_height):
    """function loads a random images from a random folder in our test path"""
    folders=list(filter(lambda x: os.path.join(path,x)), os.listdir(path))
    random_directory=np.random.randint(0,len(folders))
    path_class=folders[random_directory]
    file_path=path+path_class
    file_names=[f for f in listdir(file_path) if isfile(join(file_path,f))]
    random_file_index=np.random.randint(0,len(file_names))
    image_name=file_names[random_file_index]
    final_path=file_path+"/"+image_name
    return image.load_img(final_path,target_size=(img_width,imgheight)),final_path,path_class

#dimensions of our images
img_width, img_height= 32,32

files=[]
predictions=[]
true_labels=[]

#predicting images

for i in range(0,10):
    path="D:/fruits-360/validation/"
    img, final_path, true_label= getRandomImage(path, img_width, img_height)
    files.append(final_path)
    true_labels.append(true_label)
    x=image.img_to_array(img)
    x=x*1./255
    x=np.expand_dims(x,axis=0)
    images=np.vstack([x])
    classes=model.predict_classes(iamges,batch_size=10)
    predictions.append(classes)
    
    
for i in range(0,len(files)):
    image=cv2.imread((files[i]))
    draw_test("prediction",class_labels[predictions[i][0]],image,true_labels[i])
    cv2.waitkey(0)
    
cv2.destroyAllWindows()

TypeError: filter expected 2 arguments, got 1